In [1]:
import logging
import os

from src.avior.registry.model.config import settings
from src.avior.registry.model.registry.model_registry import ModelRegistry
from src.avior.registry.model.schemas.model_info import ModelInfo
from src.avior.registry.model.schemas.provider_info import ProviderInfo
from src.avior.registry.model.schemas.cost import ModelCost, RateLimit
from src.avior.registry.model.services.usage_service import UsageService
from src.avior.registry.model.services.model_service import ModelService

/Users/jaredquincy/foundry-technologies-avior/avior/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jaredquincy/foundry-technologies-avior/avior/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_config_path" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [2]:
from src.avior.registry.model.config import initialize_global_registry, GLOBAL_MODEL_REGISTRY
from src.avior.registry.model.services.model_service import ModelService

In [3]:
from src.avior.registry.model.registry.model_enum import OpenAIModelEnum as OME

In [4]:

# Example of using pydantic-based config or environment variables
openai_key = settings.openai_api_key or os.getenv("OPENAI_API_KEY", "")

In [5]:
anthropic_key = settings.anthropic_api_key or os.getenv("ANTHROPIC_API_KEY", "")

In [6]:
deepmind_key = settings.google_api_key or os.getenv("GOOGLE_API_KEY", "")

In [9]:
registry = ModelRegistry()

In [10]:
usage_service = UsageService()

In [11]:
svc = ModelService(registry=registry, usage_service=usage_service)

In [ ]:
# Register an OpenAI GPT-4o model
openai_info = ModelInfo(
    model_id="openai:gpt-4o",
    model_name="gpt-4o",
    cost=ModelCost(input_cost_per_thousand=0.03, output_cost_per_thousand=0.06),
    rate_limit=RateLimit(tokens_per_minute=80000, requests_per_minute=5000),
    provider=ProviderInfo(name="OpenAI", default_api_key=openai_key),
    api_key=openai_key,
)
registry.register_model(openai_info)

In [ ]:
# 1) Call the model using a string ID
response = svc(OME.GPT_4O, "Hello world!")
print("String ID response:", response.data)
usage_summary = usage_service.get_usage_summary("openai:gpt-4")
print("Usage summary (string ID):", usage_summary)


In [16]:
# 2) Call the model via the ModelService with an Enum
response = svc(OME.GPT_4O, "Hello again via enum!")
print("Enum response:", response.data)
usage_summary = usage_service.get_usage_summary(OME.GPT_4O.value)
print("Usage summary (enum):", usage_summary)

Enum response: Hello! It looks like you're interested in enums. How can I assist you with them today?
Usage summary (enum): model_name='openai:gpt-4o' total_usage=UsageStats(total_tokens=51, prompt_tokens=22, completion_tokens=29, cost_usd=0.0024000000000000002)


In [21]:
# 3) Direct usage: retrieving the model and calling it as a function
gpt4_model = svc.get_model(OME.GPT_4O)
response = gpt4_model("What is the capital of France?")
print("Direct usage response:", response.data)

Direct usage response: The capital of France is Paris.


In [25]:
# 4) Option 1 with enum repeated
response = svc(OME.GPT_4O.value, "What is the capital of Germany?")
print("Enum usage response data:", response.data)
print("Enum usage response usage:", response.usage)

Enum usage response data: The capital of Germany is Berlin.
Enum usage response usage: total_tokens=22 prompt_tokens=14 completion_tokens=8 cost_usd=0.0009


In [26]:
# 5) Option 2 with string repeated
response = svc("openai:gpt-4o", "What is the capital of Italy?")
print("String usage response data:", response.data)

String usage response data: The capital of Italy is Rome.


In [29]:
# 6) "Pytorch-like" usage (calling forward() explicitly)
response = gpt4_model("What is the capital of Spain?")
print("Forward() usage response:", response.data)

Forward() usage response: The capital of Spain is Madrid.
